In [117]:
# Importing Libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [118]:
df = pd.read_csv('data/flipkart_clean_data.csv')

In [119]:
df.head()

,Unnamed: 0.1,Unnamed: 0,id,name,price,mrp,variant,image_url,tags
0,0,0,SMPEPGKYW9GZWYVT,Clinic Plus Strong & Thick Health Shampoo,179.0,189.0,340 ml,https://rukminim1.flixcart.com/image/280/280/...,clinic plu strong & thick health shampoo 179.0...
1,1,1,BWSFH4YQYC9RSNZU,DOVE Deeply Nourishing Body Wash,360.0,360.0,800 ml,https://rukminim1.flixcart.com/image/280/280/...,dove deepli nourish bodi wash 360.0 360.0 800 ml
2,2,2,EDOEVT5UGZXJYAHU,EMAMI Healthy & Tasty Refined Rice Bran Oil P...,167.0,180.0,1 L,https://rukminim1.flixcart.com/image/280/280/...,emami healthi & tasti refin rice bran oil pouc...
3,3,3,FLRETEFHF5EK5ECT,Tata Sampann Fine Besan,19.0,64.0,500 g,https://rukminim1.flixcart.com/image/280/280/...,tata sampann fine besan 19.0 64.0 500 g
4,4,4,BFRFVEHYQQMAZXKY,Colgate Vedshakti Mouth Protect Spray,99.0,99.0,10 g,https://rukminim1.flixcart.com/image/280/280/...,colgat vedshakti mouth protect spray 99.0 99.0...


In [120]:
df.shape

(798, 9)

In [121]:
df.drop(['tags'],axis=1,inplace=True)

In [122]:
# drop dupicate records present in id column
df.drop_duplicates(subset=['id'],inplace = True)

In [123]:
df['id'].duplicated().sum()

0

In [124]:
# drop dupicate records present in image_url column
df.drop_duplicates(subset=['image_url'],inplace = True)

In [125]:
df['tags'] = df['name'] + df['variant']

In [126]:
! pip install nltk


In [127]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [128]:
# this fuction is conver word into it's root words
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [129]:
df['tags'] = df['tags'].apply(stem)

In [131]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,id,name,price,mrp,variant,image_url,tags
0,0,0,SMPEPGKYW9GZWYVT,Clinic Plus Strong & Thick Health Shampoo,179.0,189.0,340 ml,https://rukminim1.flixcart.com/image/280/280/...,clinic plu strong & thick health shampoo 340 ml
1,1,1,BWSFH4YQYC9RSNZU,DOVE Deeply Nourishing Body Wash,360.0,360.0,800 ml,https://rukminim1.flixcart.com/image/280/280/...,dove deepli nourish bodi wash 800 ml


In [132]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer( max_features=2000,stop_words="english" )

In [133]:
vectors = cv.fit_transform(df['tags']).toarray()

In [134]:
from sklearn.metrics.pairwise import cosine_similarity

In [135]:
similarity = cosine_similarity(vectors)

In [136]:
sorted(list(enumerate(similarity[1])),reverse = True , key=lambda x:x[1])[1:6]

[(694, 0.40089186286863654),
 (796, 0.40089186286863654),
 (739, 0.3779644730092272),
 (19, 0.3086066999241838),
 (35, 0.3086066999241838)]

In [137]:
def recommended(product):
    name_index = df[df['name'].str.contains(product)].index[0]
    distances = sorted(list(enumerate(similarity[name_index])),reverse = True ,key=lambda x:x[1])
    
    
    for i in distances[1:6]:
        print(df.iloc[i[0]][['name','variant','price','image_url']])
        print("----------------------------------------------------\n")
        

In [138]:
recommended('DOVE Deeply Nourishing Body Wash')

name                      DETTOL Cool Body Wash and Shower Gel
variant                                                 250 ml
price                                                    160.0
image_url     https://rukminim1.flixcart.com/image/280/280/...
Name: 694, dtype: object
----------------------------------------------------

name                   DETTOL Refresh Body Wash and Shower Gel
variant                                                 250 ml
price                                                    160.0
image_url     https://rukminim1.flixcart.com/image/280/280/...
Name: 796, dtype: object
----------------------------------------------------

name                NIVEA Body Wash
variant                       239.0
price         Fresh Pure Shower Gel
image_url                    500 ml
Name: 739, dtype: object
----------------------------------------------------

name                              DOVE Daily Shine Conditioner
variant                                             

In [139]:
import pickle